In [1]:
%ls

00284.jpg*                inference.ipynb*                   requirements.txt*
convert_to_lite.ipynb*    lite_fire_detection_model.tflite*  saved_model/
data/                     mlflowconfig.ipynb*                train.ipynb*
features.txt*             mlruns/                            validation.ipynb*
fire_detection/           preprocess.ipynb*
fire_detection.pipeline*  report.txt*


In [24]:
!pip3 install tensorflow==2.3.0 tensorflow_hub==0.10.0 matplotlib numpy mlflow==1.10.0 minio boto3==1.9.240 opencv-python==4.4.0.46

     |████████████████████████████████| 49.5 MB 6.2 MB/s eta 0:00:01   |                                | 20 kB 1.4 MB/s eta 0:00:37     |▏                               | 286 kB 783 kB/s eta 0:01:03     |█▋                              | 2.5 MB 783 kB/s eta 0:01:00     |██                              | 3.2 MB 783 kB/s eta 0:01:00     |██████████████▏                 | 22.0 MB 5.9 MB/s eta 0:00:05     |███████████████████▋            | 30.3 MB 2.7 MB/s eta 0:00:08     |███████████████████▊            | 30.5 MB 2.7 MB/s eta 0:00:08     |████████████████████▉           | 32.2 MB 2.7 MB/s eta 0:00:07     |██████████████████████▏         | 34.3 MB 1.0 MB/s eta 0:00:15     |██████████████████████▍         | 34.6 MB 1.0 MB/s eta 0:00:15     |███████████████████████         | 35.6 MB 2.0 MB/s eta 0:00:07     |███████████████████████▍        | 36.2 MB 2.0 MB/s eta 0:00:07      | 36.5 MB 2.0 MB/s eta 0:00:07████████████████████████      | 40.2 MB 5.2 MB/s eta 0:00:02     |█████████████████████

In [39]:
!pip freeze | grep opencv-python

opencv-python==4.4.0.46


In [3]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import boto3
import mlflow
from minio import Minio
from minio.error import ResponseError
import json

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF version: 2.3.0
Hub version: 0.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [13]:
def lite_model(image):
  # Load the TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path="lite_fire_detection_model.tflite")
  
  interpreter.allocate_tensors()
  interpreter.set_tensor(interpreter.get_input_details()[0]['index'], image)
  interpreter.invoke()
  return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

In [36]:
LABELS=['nessun incendio', 'incendio']

In [30]:
from PIL import Image
image = Image.open('200284_resized.jpg')
data = np.asarray(image, dtype="float32" )
data.shape

(224, 224, 3)

In [28]:
#resize the image

In [31]:
probs_lite = lite_model(data[None, ...])[0]

In [37]:
predicted_index = np.argmax(probs_lite)
LABELS[predicted_index]

'incendio'